In [74]:
import sys 
import os 
import networkx as nx
from pyvis.network import Network


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.getLatestFiles import get_latest_master_players, get_latest_master_matches, get_latest_master_events


In [38]:
master_players_df = get_latest_master_players()
master_matches_df = get_latest_master_matches()
master_events_df = get_latest_master_events()



✅ 2697 players found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Players/20251115_master_players.csv
✅ 23813 matches found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Matches/20251117_master_matches.csv 
✅ 187 events found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Events/20251115_master_events.csv 


In [42]:
master_matches_df

,eventId,documentCode,subEventName,subEventDescription,venueName,tableName,dnf,ttrReview,serverNext,duration (unreliable),...,loser0Wins,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet,Round
0,2410,TTEMSINGLES-----------RND1000700--,Men Singles,Men's Singles - Preliminary Round 1 - Match 7,Lusail Sports Arena,Table 6,False,NaN,NaN,00:15:15,...,0.0,0.0,0.0,2.33,0.0,False,False,12.00,9.67,Other
1,2410,TTEMSINGLES-----------RND1002600--,Men Singles,Men's Singles - Preliminary Round 1 - Match 26,Lusail Sports Arena,Table 4,False,NaN,NaN,00:00:21,...,0.0,0.0,0.0,5.67,0.0,False,False,11.33,5.67,Other
2,2410,TTEMSINGLES-----------RND1004600--,Men Singles,Men's Singles - Preliminary Round 1 - Match 46,Lusail Sports Arena,Table 3,False,NaN,NaN,00:00:00,...,0.0,0.0,0.0,2.33,0.0,False,False,11.00,8.67,Other
3,2410,TTEMSINGLES-----------RND1001900--,Men Singles,Men's Singles - Preliminary Round 1 - Match 19,Lusail Sports Arena,Table 5,False,NaN,NaN,00:27:19,...,0.0,0.0,0.0,3.00,0.0,False,False,12.33,9.33,Other
4,2410,TTEMSINGLES-----------RND1001000--,Men Singles,Men's Singles - Preliminary Round 1 - Match 10,Lusail Sports Arena,Table 7,False,NaN,NaN,00:13:28,...,0.0,0.0,0.0,3.67,0.0,False,False,12.00,8.33,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23808,3191,TTEMSINGLES-----------QFNL000100----------,Men's Singles,Men's Singles - Quarterfinal - Match 1,Centrum Szkolenia PZTS,Table 1,False,False,101192.0,00:38:46,...,0.0,0.0,1.0,4.67,5.0,False,False,9.00,8.20,Quarterfinal
23809,3191,TTEWSINGLES-----------SFNL000100----------,Women's Singles,Women's Singles - Semifinal - Match 1,Centrum Szkolenia PZTS,Table 1,False,False,122261.0,00:19:01,...,0.0,0.0,0.0,6.67,0.0,False,False,11.00,4.33,Semifinal
23810,3191,TTEWSINGLES-----------SFNL000200----------,Women's Singles,Women's Singles - Semifinal - Match 2,Centrum Szkolenia PZTS,Table 1,False,False,135370.0,00:41:40,...,0.0,0.0,2.0,3.33,7.5,False,False,8.40,9.40,Semifinal
23811,3191,TTEMSINGLES-----------SFNL000200----------,Men's Singles,Men's Singles - Semifinal - Match 2,Centrum Szkolenia PZTS,Table 1,False,False,116818.0,00:32:30,...,0.0,0.0,0.0,2.67,4.0,False,False,10.00,9.00,Semifinal


In [47]:
master_matches_df["matchupIds"] = master_matches_df.apply(
    lambda x: tuple(sorted((x["winnerId"], x["loserId"]))), 
    axis=1
)

In [ ]:
men_players_df = master_players_df[master_players_df['Gender'] == 'M']
men_player_ids = set(men_players_df['playerId'])
men_matches_filter = master_matches_df['winnerId'].isin(men_player_ids) & master_matches_df['loserId'].isin(men_player_ids)
men_matches_df = master_matches_df[men_matches_filter]


women_players_df = master_players_df[master_players_df['Gender'] == 'F']
women_player_ids = set(women_players_df['playerId'])
women_matches_filter = master_matches_df['winnerId'].isin(women_player_ids) & master_matches_df['loserId'].isin(women_player_ids)
women_matches_df = master_matches_df[women_matches_filter]

In [48]:
G = nx.from_pandas_edgelist(master_matches_df, "winnerId", "loserId")

In [49]:
components = list(nx.connected_components(G))

In [50]:
print(f"--- Men's Network Analysis ---")
print(f"Total Men Players (Nodes): {G.number_of_nodes()}")
print(f"Total Matchups (Edges): {G.number_of_edges()}")
print(f"Number of 'Islands' (Disconnected Components): {len(components)}")

--- Men's Network Analysis ---
Total Men Players (Nodes): 2697
Total Matchups (Edges): 20026
Number of 'Islands' (Disconnected Components): 8


In [80]:
components_lengths = [ len(component) for component in components ]
components_lengths

[1529, 1156, 2, 2, 2, 2, 2, 2]

In [56]:
for index, component in enumerate(components):
    cpt_df = master_players_df[master_players_df["playerId"].isin(component)]
    cpt_df = cpt_df.sort_values(by="playerId")
    cpt_df = cpt_df.reset_index(drop=True)
    cpt_df.to_csv(f"./players_cpt_{index}.csv", index=False)
    

In [64]:
import itertools
small_cpts = [cpt for cpt in components if len(cpt) < 10]
small_cpt_ids = list(set(list(itertools.chain.from_iterable(small_cpts))))
small_cpt_ids

[200363,
 145484,
 200365,
 131791,
 131472,
 133265,
 137427,
 136503,
 135288,
 111964,
 123838,
 110879]

In [65]:
home_small_cpt = master_matches_df["winnerId"].isin(small_cpt_ids)
away_small_cpt = master_matches_df["loserId"].isin(small_cpt_ids)

small_cpt_filter = home_small_cpt | away_small_cpt
master_matches_df[small_cpt_filter]

,eventId,documentCode,subEventName,subEventDescription,venueName,tableName,dnf,ttrReview,serverNext,duration (unreliable),...,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet,Round,matchupIds
844,2473,TTEWSINGLES-----------R64-000200----------,Women Singles,Women's Singles - Round of 64 - Match 2,Versatile Sports Palace in Yaounde,Table 1,False,False,NaN,00:30:00,...,0.0,0.0,3.5,2.0,False,False,10.8,8.4,Other,"(136503, 137427)"
847,2473,TTEWSINGLES-----------R64-001000----------,Women Singles,Women's Singles - Round of 64 - Match 10,Versatile Sports Palace in Yaounde,Table 6,False,False,NaN,00:30:00,...,0.0,0.0,7.0,0.0,False,False,11.0,4.0,Other,"(131791, 200363)"
848,2473,TTEWSINGLES-----------R64-002200----------,Women Singles,Women's Singles - Round of 64 - Match 22,Versatile Sports Palace in Yaounde,Table 6,WO,False,NaN,00:00:00,...,0.0,0.0,11.0,0.0,False,False,11.0,0.0,Other,"(133265, 135288)"
850,2473,TTEWSINGLES-----------R64-002300----------,Women Singles,Women's Singles - Round of 64 - Match 23,Versatile Sports Palace in Yaounde,Table 7,False,False,NaN,00:30:00,...,0.0,0.0,4.0,0.0,False,False,11.5,7.5,Other,"(145484, 200365)"
853,2473,TTEWSINGLES-----------R64-001800----------,Women Singles,Women's Singles - Round of 64 - Match 18,Versatile Sports Palace in Yaounde,Table 2,False,False,NaN,00:30:00,...,0.0,0.0,5.5,3.0,False,False,10.4,6.6,Other,"(111964, 123838)"
859,2473,TTEWSINGLES-----------R64-003000----------,Women Singles,Women's Singles - Round of 64 - Match 30,Versatile Sports Palace in Yaounde,Table 3,WO,False,NaN,00:00:00,...,0.0,0.0,11.0,0.0,False,False,11.0,0.0,Other,"(110879, 131472)"


In [ ]:
africa_cup_filter = master_matches_df["eventId"] == 2473
women_singles_filter = master_matches_df["subEventName"] == "Women Singles"
 
master_matches_df[africa_cup_filter & women_singles_filter]

,eventId,documentCode,subEventName,subEventDescription,venueName,tableName,dnf,ttrReview,serverNext,duration (unreliable),...,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet,Round,matchupIds
840,2473,TTEWSINGLES-----------R64-001200----------,Women Singles,Women's Singles - Round of 64 - Match 12,Versatile Sports Palace in Yaounde,Table 8,WO,False,NaN,00:00:00,...,0.0,0.0,11.00,0.00,False,False,11.00,0.00,Other,"(112897, 136516)"
841,2473,TTEWSINGLES-----------R64-000300----------,Women Singles,Women's Singles - Round of 64 - Match 3,Versatile Sports Palace in Yaounde,Table 2,False,False,NaN,00:30:00,...,0.0,0.0,7.75,0.00,False,False,11.00,3.25,Other,"(122195, 200409)"
842,2473,TTEWSINGLES-----------R64-000700----------,Women Singles,Women's Singles - Round of 64 - Match 7,Versatile Sports Palace in Yaounde,Table 5,False,False,NaN,00:30:00,...,0.0,0.0,5.75,4.00,False,False,10.20,6.40,Other,"(136089, 200382)"
843,2473,TTEWSINGLES-----------R64-000500----------,Women Singles,Women's Singles - Round of 64 - Match 5,Versatile Sports Palace in Yaounde,Table 3,WO,False,NaN,00:00:00,...,0.0,0.0,11.00,0.00,False,False,11.00,0.00,Other,"(112900, 115379)"
844,2473,TTEWSINGLES-----------R64-000200----------,Women Singles,Women's Singles - Round of 64 - Match 2,Versatile Sports Palace in Yaounde,Table 1,False,False,NaN,00:30:00,...,0.0,0.0,3.50,2.00,False,False,10.80,8.40,Other,"(136503, 137427)"
845,2473,TTEWSINGLES-----------R64-000600----------,Women Singles,Women's Singles - Round of 64 - Match 6,Versatile Sports Palace in Yaounde,Table 4,False,False,NaN,00:30:00,...,0.0,1.0,4.00,6.00,False,False,9.00,8.33,Other,"(131473, 131741)"
846,2473,TTEWSINGLES-----------R64-001100----------,Women Singles,Women's Singles - Round of 64 - Match 11,Versatile Sports Palace in Yaounde,Table 7,False,False,NaN,00:30:00,...,0.0,0.0,7.50,0.00,False,False,11.00,3.50,Other,"(131335, 145596)"
847,2473,TTEWSINGLES-----------R64-001000----------,Women Singles,Women's Singles - Round of 64 - Match 10,Versatile Sports Palace in Yaounde,Table 6,False,False,NaN,00:30:00,...,0.0,0.0,7.00,0.00,False,False,11.00,4.00,Other,"(131791, 200363)"
848,2473,TTEWSINGLES-----------R64-002200----------,Women Singles,Women's Singles - Round of 64 - Match 22,Versatile Sports Palace in Yaounde,Table 6,WO,False,NaN,00:00:00,...,0.0,0.0,11.00,0.00,False,False,11.00,0.00,Other,"(133265, 135288)"
849,2473,TTEWSINGLES-----------R64-002000----------,Women Singles,Women's Singles - Round of 64 - Match 20,Versatile Sports Palace in Yaounde,Table 4,False,False,NaN,00:30:00,...,0.0,0.0,8.25,0.00,False,False,11.00,2.75,Other,"(136521, 146071)"


In [71]:
output_dir = "../Data/NetworkVisuals/"
os.makedirs(output_dir, exist_ok=True)

In [76]:
mens_component = components[0]
womens_component = components[1]

In [113]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import os

# --- 1. CRITICAL: Fix Data Types Before Building Graph ---
# This ensures the 'playerId' keys match.
try:
    master_matches_df['winnerId'] = pd.to_numeric(master_matches_df['winnerId'])
    master_matches_df['loserId'] = pd.to_numeric(master_matches_df['loserId'])
    master_players_df['playerId'] = pd.to_numeric(master_players_df['playerId'])
    print("✅ ID columns standardized to numeric type.")
except Exception as e:
    print(f"⚠️ Warning: Could not convert all IDs to numeric. {e}")

# --- 2. Build Your Graph & Get Components (as you did) ---
G = nx.from_pandas_edgelist(master_matches_df, "winnerId", "loserId")
all_components = sorted(nx.connected_components(G), key=len, reverse=True)

# Get the two main components
nodes_component_1 = all_components[0]
nodes_component_2 = all_components[1]
G_men = G.subgraph(nodes_component_1)
G_women = G.subgraph(nodes_component_2)

# --- 3. Define the FINAL Visualization Function ---
def visualize_top_component(graph, top_n, player_df, filename, heading):
    """
    Finds the Top N most-connected players, adds player metadata,
    and saves an interactive visualization with CIRCULAR image nodes.
    """
    print(f"\n--- Processing: {heading} ---")
    
    degrees = dict(graph.degree())
    sorted_players = sorted(degrees.items(), key=lambda item: item[1], reverse=True)
    top_player_ids = [player_id for player_id, degree in sorted_players[:top_n]]
    
    G_filtered = graph.subgraph(top_player_ids)
    
    # --- 3. Add Player Metadata to Nodes ---
    player_map = player_df.set_index('playerId').to_dict('index')
    
    titles = {}
    shapes = {}
    images = {}
    
    for node in G_filtered.nodes():
        player_info = player_map.get(node)
        
        # Set Title (for hover/click)
        if player_info:
            player_name = player_info.get('PlayerName', 'Unknown')
            player_country = player_info.get('CountryName', 'N/A')
            titles[node] = f"{player_name} ({player_country}) | Connections: {degrees.get(node)}"
        else:
            titles[node] = f"Unknown ID: {node} | Connections: {degrees.get(node)}"

        # --- NEW: Set Node Shape & Image (The Flag) ---
        flag_url = player_info.get('flagUrl')
        if player_info and pd.notna(flag_url):
            shapes[node] = 'circularImage'  # <-- 1. SETS SHAPE TO CIRCLE
            images[node] = flag_url
        else:
            shapes[node] = 'dot' 
            images[node] = '' # Ensure no image is passed

    # Set the node attributes in the graph
    nx.set_node_attributes(G_filtered, titles, 'title')
    nx.set_node_attributes(G_filtered, shapes, 'shape')
    nx.set_node_attributes(G_filtered, images, 'image')

    print(f"Filtered {heading} Network: {G_filtered.number_of_nodes()} nodes, {G_filtered.number_of_edges()} edges")

    # --- 4. Visualize with pyvis ---
    net = Network(
        height="800px", 
        width="100%", 
        notebook=True,
        heading=heading,
        bgcolor="#222222",
        font_color="white"
    )
    
    # --- 5. NEW: Set Options (Physics & Interaction) ---
    net.set_options("""
    {
      "nodes": {
        "borderWidth": 3,
        "size": 25, 
        "shapeProperties": {
          "useImageSize": false, 
          "useBorderWithImage": true
        },
        "font": {
          "size": 16,
          "color": "white"
        }
      },
      "edges": {
        "width": 0.15,
        "color": { "inherit": "from" },
        "smooth": { "type": "continuous" }
      },
      "physics": {
        "enabled": true,
        "barnesHut": {
          "gravitationalConstant": -20000, 
          "centralGravity": 0.1,
          "springLength": 200, 
          "springConstant": 0.05,
          "damping": 1,
          "avoidOverlap": 0.1
        },
        "solver": "barnesHut",
        "stabilization": {
          "iterations": 100,
          "fit": true
        }
      },
      "interaction": {
        "hover": true,
        "hoverConnectedEdges": true,
        "selectConnectedEdges": true,
        "navigationButtons": true,
        "keyboard": true,
        "hideEdgesOnDrag": true,
        "hideNodesOnDrag": true
      }
    }
    """)
    
    net.from_nx(G_filtered)
    
    # Save the file
    output_filename = f"{filename}_top{top_n}.html"
    net.show(output_filename)
    print(f"✅ Interactive graph saved to {output_filename}")

# --- 6. Run the Visualization ---
# (Your code to run visualize_top_component goes here)

# --- 6. Run the Visualization ---
TOP_N_PLAYERS = 100
    
visualize_top_component(
    G_men, 
    TOP_N_PLAYERS, 
    master_players_df,
    "men_network", 
    f"Men's Top {TOP_N_PLAYERS} Player Network"
)
    
visualize_top_component(
    G_women, 
    TOP_N_PLAYERS, 
    master_players_df,
    "women_network", 
    f"Women's Top {TOP_N_PLAYERS} Player Network"
)

✅ ID columns standardized to numeric type.

--- Processing: Men's Top 100 Player Network ---
Filtered Men's Top 100 Player Network Network: 100 nodes, 2081 edges
men_network_top100.html
✅ Interactive graph saved to men_network_top100.html

--- Processing: Women's Top 100 Player Network ---
Filtered Women's Top 100 Player Network Network: 100 nodes, 2100 edges
women_network_top100.html
✅ Interactive graph saved to women_network_top100.html


In [96]:
master_players_df.columns

Index(['playerId', 'PlayerName', 'PlayerFamilyName', 'PlayerGivenName',
       'PlayerFamilyNameFirst', 'Gender', 'Age', 'DOB', 'CountryName',
       'CountryCode', 'NationalityName', 'NationalityCode', 'OrganizationName',
       'OrganizationCode', 'Style', 'Grip', 'Hand', 'HeadShot', 'flagUrl'],
      dtype='object')

In [97]:
master_matches_df.columns

Index(['eventId', 'documentCode', 'subEventName', 'subEventDescription',
       'venueName', 'tableName', 'dnf', 'ttrReview', 'serverNext',
       'duration (unreliable)', 'EventName', 'EventStartDate', 'matchDate',
       'overallScore', 'gameScore', 'Winner', 'bestOf', 'winnerName',
       'winnerId', 'winnerCountry', 'loserName', 'loserId', 'loserCountry',
       'winnerSets', 'loserSets', 'totalSets', 'isStraightSets', 'isDecider',
       'winnerTotalPoints', 'loserTotalPoints', 'totalPoints',
       'winnerPointsRatio', 'loserPointsRatio', 'winnerMaxScore',
       'loserMaxScore', 'maxScore', 'numberDeuceGames', 'winnerDeuceWon',
       'loserDeuceWon', 'winnerPointsDifference', 'pointsPerSet',
       'winnerDroppedFirstSet', 'winner0Wins', 'winner1Wins', 'winner4Wins',
       'loser0Wins', 'loser1Wins', 'loser4Wins', 'winnerAvgWinningMargin',
       'loserAvgWinningMargin', 'comebackBy2', 'comebackBy3',
       'winnerAvgPointsPerSet', 'loserAvgPointsPerSet', 'Round', 'matchupIds'